<a href="https://colab.research.google.com/github/1hamzaiqbal/MFCLIP_acv/blob/hamza%2Fdiscrim/mf_clip_finetune_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 0) GPU + repo
!nvidia-smi
%cd /content
!git clone -b hamza/discrim https://github.com/1hamzaiqbal/MFCLIP_acv
%cd MFCLIP_acv

In [2]:
# 1) Deps
!pip install torch torchvision timm einops yacs tqdm opencv-python scikit-learn scipy pyyaml ruamel.yaml pytorch-ignite foolbox pandas matplotlib seaborn wilds ftfy timm



In [3]:
# 2) Data (example: Oxford Pets into /content/data/oxford_pets)
from torchvision.datasets import OxfordIIITPet
from torchvision import transforms
from pathlib import Path
root = Path("/content/data/oxford_pets")
root.mkdir(parents=True, exist_ok=True)
_ = OxfordIIITPet(root=str(root), download=True, transform=transforms.ToTensor())
print("Oxford Pets downloaded to", root)



In [4]:
# fetch official Oxford-IIIT Pet dataset (provides annotations/trainval.txt)
%cd /content
!mkdir -p /content/data/oxford_pets
!wget -q https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget -q https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
!tar -xf images.tar.gz -C /content/data/oxford_pets
!tar -xf annotations.tar.gz -C /content/data/oxford_pets
!ls /content/data/oxford_pets/annotations | head


In [5]:
# 3) Finetune Surrogate (Required for generator training)
# go to the repo root first
%cd /content/MFCLIP_acv

# (optional) quiet TF spam
%env TF_CPP_MIN_LOG_LEVEL=2

# run with repo-relative config paths (no MFCLIP_acv/ prefix)
!python main.py \
  --flag finetune \
  --dataset oxford_pets \
  --root /content/data \
  --config-file configs/trainers/CoOp/rn50.yaml \
  --dataset-config-file configs/datasets/oxford_pets.yaml \
  --trainer ZeroshotCLIP \
  --surrogate RN50 \
  --head ArcFace \
  --num_epoch 1 \
  --bs 64 \
  --lr 0.01 \
  --optimizer SGD \
  --ratio 0.2 \
  --device cuda:0


In [6]:
# 4) Train ViT Generator
!python main.py \
  --flag train_unet \
  --generator vit \
  --dataset oxford_pets \
  --root /content/data \
  --config-file configs/trainers/CoOp/rn50.yaml \
  --dataset-config-file configs/datasets/oxford_pets.yaml \
  --trainer ZeroshotCLIP \
  --surrogate RN50 \
  --head ArcFace \
  --num_epoch 1 \
  --bs 64 \
  --lr 0.01 \
  --optimizer SGD \
  --ratio 0.2 \
  --device cuda:0
